# clean data

In [18]:
import pandas as pd 
import numpy as np 

ufos = pd.read_csv('1-Web-App/data/ufos.csv')
print(ufos.head())
ufos.info()

           datetime                  city state country     shape  \
0  10/10/1949 20:30            san marcos    tx      us  cylinder   
1  10/10/1949 21:00          lackland afb    tx     NaN     light   
2  10/10/1955 17:00  chester (uk/england)   NaN      gb    circle   
3  10/10/1956 21:00                  edna    tx      us    circle   
4  10/10/1960 20:00               kaneohe    hi      us     light   

   duration (seconds) duration (hours/min)  \
0              2700.0           45 minutes   
1              7200.0              1-2 hrs   
2                20.0           20 seconds   
3                20.0             1/2 hour   
4               900.0           15 minutes   

                                            comments date posted   latitude  \
0  This event took place in early fall around 194...   4/27/2004  29.883056   
1  1949 Lackland AFB&#44 TX.  Lights racing acros...  12/16/2005  29.384210   
2  Green/Orange circular disc over Chester&#44 En...   1/21/2008  53.20

In [19]:
# chon nhan phu hop tu bang de tao bang moi
ufos = pd.DataFrame({'Seconds': ufos['duration (seconds)'], 'Country': ufos['country'],'Latitude': ufos['latitude'],'Longitude': ufos['longitude']})

ufos.Country.unique()
ufos.head()

,Seconds,Country,Latitude,Longitude
0,2700.0,us,29.883056,-97.941111
1,7200.0,NaN,29.384210,-98.581082
2,20.0,gb,53.200000,-2.916667
3,20.0,us,28.978333,-96.645833
4,900.0,us,21.418056,-157.803611


In [20]:
# reduce null values and  import sightings between 1-60 seconds
ufos.dropna(inplace=True)
ufos = ufos[(ufos['Seconds']>=1) & (ufos['Seconds']<=60)]
ufos.head()

,Seconds,Country,Latitude,Longitude
2,20.0,gb,53.200000,-2.916667
3,20.0,us,28.978333,-96.645833
14,30.0,us,35.823889,-80.253611
23,60.0,us,45.582778,-122.352222
24,3.0,gb,51.783333,-0.783333


In [21]:
# import scikit-learn
from sklearn.preprocessing import LabelEncoder
ufos['Country'] = LabelEncoder().fit_transform(ufos['Country'])
ufos.head()

,Seconds,Country,Latitude,Longitude
2,20.0,3,53.200000,-2.916667
3,20.0,4,28.978333,-96.645833
14,30.0,4,35.823889,-80.253611
23,60.0,4,45.582778,-122.352222
24,3.0,3,51.783333,-0.783333


# build model

In [25]:
from sklearn.model_selection import train_test_split
X = ufos[['Seconds','Latitude','Longitude']]
y = ufos['Country']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
# train and test
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
model = LogisticRegression()
model.fit(X_train, y_train)


predictions = model.predict(X_test)
print(classification_report(y_test, predictions))
print(f'accuracy: {accuracy_score(y_test,predictions)}')

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       0.78      0.17      0.28       288
           2       1.00      0.90      0.95        10
           3       0.99      1.00      1.00       134
           4       0.95      1.00      0.97      4700

    accuracy                           0.95      5173
   macro avg       0.94      0.81      0.84      5173
weighted avg       0.94      0.95      0.94      5173

accuracy: 0.9508988981248792


/home/dat/anaconda3/envs/tracking-barbell-exercises/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# save pickle model

In [30]:
import pickle
model_fliename = 'ufo_model.pkl'
pickle.dump(model, open(model_fliename,'wb'))

model = pickle.load(open(model_fliename,'rb'))
print(model.predict([[20.0,	53.200000,	-2.916667]]))

[3]


/home/dat/anaconda3/envs/tracking-barbell-exercises/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


# build a Flask app